### Calculate the reagent volume state space from the concentration state space (the reaction pool). With the volume state space, we can easily find the volumes of reagents for each AL iteration.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from Data.datasets import save_obj, load_obj
import itertools

# load the homogeneous concentration space
R_conc = load_obj('8R homogeneous concentration statespace (Pb2, morph, H2O and FAH constrained)')
R_vol = pd.DataFrame(columns = ['R1','R2','R3','R4','R5','R6','R7'], index = R_conc.index)

In [ ]:
R_vol['R6'] = R_conc['FAH']*(300/1000)*(46/1.22) # R6 is the volume of formic acid
R_vol['R7'] = R_conc['H2O']*(300/1000)*(18/0.998) # R7 is the volume of water
R_vol['R4'] = (R_conc['Pb']*300)/2.32 # R4 is the volume of lead iodide+morpholinium iodide DMF solution
R_vol['R5'] = ((R_conc['morph']*300)-(R_vol['R4']*2.91))/2.36 # R5 is the volume of morpholinium iodide only DMF solution
R_vol['R1+R2+R3'] = 300-(R_vol['R4']+R_vol['R5']+R_vol['R6']+R_vol['R7']) 
R_vol['R2'] = R_conc['DMSO']*(R_vol['R1+R2+R3']+(0.494*R_vol['R4'])+(0.731*R_vol['R5'])) # R2 is the volume of pure DMSO
R_vol['R3'] = R_conc['GBL']*(R_vol['R1+R2+R3']+(0.494*R_vol['R4'])+(0.731*R_vol['R5'])) # R3 is the volume of pure GBL
R_vol['R1'] = R_vol['R1+R2+R3']-R_vol['R2']-R_vol['R3'] # R1 is the volume of pure DMF
R_vol = R_vol.drop(['R1+R2+R3'], axis=1)
R_vol = round(R_vol)
####################
print('Are all volumes positive values?', (R_vol >= 0).all().all()) # sanity check

In [ ]:
# Now back calculate the concentration space to validate concentration-volume conversion
# Below are concentrations for each reagent: _a is PbI2, _b is morph, _c is solvent vol fraction (e.g. DMSO / DMSO solution)
############
R4_a = 2.32
R4_b = 2.91
R4_c = 0.494
############
R5 = 2.36
R5_c = 0.731
############
R_conc_val = pd.DataFrame()

R_conc_val['Pb'] = (R_vol['R4']*R4_a)/300

R_conc_val['morph'] = (R_vol['R4']*R4_b + R_vol['R5']*R5)/300

# Total volume of solvents...it is NOT equal to the total volume of solution, which is set to 300 uL.
totvol_solv = R_vol['R1'] + R_vol['R2'] + R_vol['R3']\
              + R_vol['R4']*R4_c + R_vol['R5']*R5_c

# solvent/solvents volume fraction: e.g., vol[DMSO]/(vol)
R_conc_val['DMSO'] = R_vol['R2']/totvol_solv
R_conc_val['GBL'] = R_vol['R3']/totvol_solv

R_conc_val['FAH'] = ((R_vol['R6']*1.22)/46)/(300/1000)
R_conc_val['H2O'] = ((R_vol['R7']*0.998)/18)/(300/1000)

R_conc_val = R_conc_val.fillna(0)

err_conc = R_conc_val-R_conc

In [ ]:
print('Pb conc error in M', max(err_conc['Pb']))
print('morph conc error in M', max(err_conc['morph']))
print('DMSO conc error in fraction', max(err_conc['DMSO']))
print('GBL conc error in fraction', max(err_conc['GBL']))
print('formic acid conc error in M', max(err_conc['FAH']))
print('water conc error in M', max(err_conc['H2O']))

In [ ]:
save_obj(R_vol, '8R homogeneous volume statespace (Pb2, morph, H2O and FAH constrained)')
save_obj(err_conc, '8R homogeneous concentration statespace_error(from volume) (Pb2, morph, H2O and FAH constrained)')